In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
!pip install lightgbm --install-option=--gpu
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, mean_squared_error

In [ ]:
!ls ../input/tabular-playground-series-apr-2021/

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")
sample = pd.read_csv("../input/tabular-playground-series-apr-2021/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
test_id = test['PassengerId']
train.drop('PassengerId', inplace = True, axis = 1)
test.drop('PassengerId', inplace = True, axis = 1)

In [ ]:
train['First_name'] = [i.split(',')[0] for i in train['Name']]
train['Last_name'] = [i.split(',')[1] for i in train['Name']]

test['First_name'] = [i.split(',')[0] for i in test['Name']]
test['Last_name'] = [i.split(',')[1] for i in test['Name']]

train.drop('Name', inplace = True, axis = 1)
test.drop('Name', inplace = True, axis = 1)

In [ ]:
train.head()

In [ ]:
train.isnull().sum() / len(train) * 100

In [ ]:
test.isnull().sum() / len(train) * 100

In [ ]:
for i in train.columns:
    if train[i].isnull().sum().any():
        print(i)
        print(train[i].value_counts())
        print("*******"*6)

In [ ]:
train.info()

In [ ]:
for i in train.columns:
    if train[i].isnull().sum().any():
        if train[i].dtype == 'object':
            train[i] = train[i].fillna(train[i].mode()[0])
            test[i] = test[i].fillna(test[i].mode()[0])
        else:
            train[i] = train[i].fillna(train[i].mean())
            test[i] = test[i].fillna(test[i].mean())

In [ ]:
train.isnull().sum().any()

In [ ]:
test.head()

In [ ]:
train['Cabin_name'] = [i[0] for i in train['Cabin']]
train['Cabin_number'] = [i[1:] for i in train['Cabin']]

test['Cabin_name'] = [i[0] for i in test['Cabin']]
test['Cabin_number'] = [i[1:] for i in test['Cabin']]

train.drop('Cabin', inplace = True, axis = 1)
test.drop('Cabin', inplace = True, axis = 1)

In [ ]:
train.head()

In [ ]:
print(train['Survived'].value_counts() / len(train) * 100)
plt.figure(figsize = (8,5))
sns.countplot(x = train['Survived'])
plt.xlabel('Survived', size = 12)
plt.ylabel('Count', size = 12)
plt.title('Distribution of class in target column', size = 12)

*Target column is not that much imbalanced so we will leave it as it is.*

In [ ]:
la = LabelEncoder()
for i in train.columns:
    if train[i].dtype == 'object':
        train[i] = la.fit_transform(train[i])
        test[i] = la.fit_transform(test[i])
train.info()

In [ ]:
train.describe()

In [ ]:
train.drop(['First_name', 'Last_name', 'Ticket'], inplace = True, axis = 1)
test.drop(['First_name', 'Last_name', 'Ticket'], inplace = True, axis = 1)

In [ ]:
x = train.iloc[:, 1:].copy()
col = x.columns

std = StandardScaler()
x_std = std.fit_transform(x)
x_std = pd.DataFrame(data = x_std, columns = col)

mms = MinMaxScaler()
x_mms = mms.fit_transform(x)
x_mms = pd.DataFrame(data = x_mms, columns = col)

In [ ]:
train.head()

In [ ]:
y = train.iloc[:, 0].copy()
y.head()

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [ ]:
def Models(model, name, d, xtrain, ytrain, xtest, ytest):
    print("Working on {} model".format(name))
    
    cla = model
    cla.fit(xtrain, ytrain)
    
    predicted = cla.predict(xtrain)
    tr_auc = accuracy_score(predicted, ytrain)*100
#     print("Print accuracy on Training data is {}".format(tr_auc))
    
    predicted = cla.predict(xtest)
    te_auc = accuracy_score(predicted, ytest)*100

#     Recall = recall_score(predicted, ytest, average = 'micro')
    
#     Precision = precision_score(predicted, ytest, average = 'micro')
    
    F1 = f1_score(predicted, ytest, average = 'micro')
    
    MSE = mean_squared_error(predicted, ytest)
    
    d['Name'].append(name)
    d['Training ACU'].append(tr_auc)
    d['Testing ACU'].append(te_auc)
    d['F1_Score'].append(F1)
#     d['Recall'].append(Recall)
#     d['Precision'].append(Precision)
    d['MSE'].append(MSE)
    
    print("**********"*5)
    print()
    return d

In [ ]:
l = [x, x_std, x_mms]
final = []
for i in l:
    xtrain, xtest, ytrain, ytest = train_test_split(i, y, test_size = 0.25, random_state = 42)
    
    d = {'Name' : [], 'Training ACU': [], 'Testing ACU': [], 'F1_Score': [], 'MSE': []}
    
    models = [ 
        [RandomForestClassifier(n_estimators = 350), 'Random Forest'], [DecisionTreeClassifier(), 'Decision Tree'], 
              [XGBClassifier(tree_method='gpu_hist'), 'XGBoost'], [LGBMClassifier(device_type = 'gpu'), 'LightGBM'], 
              [CatBoostClassifier(task_type="GPU"), 'CatBoost'], [GaussianNB(), 'Naive Bayes'], 
              [LogisticRegression(), 'Logistic Regression'] 
    ]

    for model in models:
        d = Models(model[0], model[1], d, xtrain, ytrain, xtest, ytest)
    final.append(d)

In [ ]:
name = ['Normal', 'Standard', 'Min Max']
for i in range (len(name)):
    print(name[i])
    acu_data = pd.DataFrame(data = final[i])
    print(acu_data)
    print("******"*12)

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x_std, y, test_size = 0.25, random_state = 42)

cla = LGBMClassifier(device_type="GPU")
cla.fit(xtrain, ytrain, eval_set = (xtest, ytest), verbose = 5, early_stopping_rounds = 5)

In [ ]:
predicted = cla.predict(xtrain)
print("Accuracy on training data is {}".format(accuracy_score(ytrain, predicted)))

predicted = cla.predict(xtest)
print("Accuracy on testing data is {}".format(accuracy_score(ytest, predicted)))

In [ ]:
x_test = test.loc[:, :]
x_test = std.fit_transform(x_test)
x_test = pd.DataFrame(data = x_test, columns = col)
pred_test = cla.predict(x_test)
pred_test

In [ ]:
sample.head()

In [ ]:
submit = pd.DataFrame(data = {'PassengerId': test_id, 'Survived': pred_test})

In [ ]:
submit.to_csv('submission1', index = False)
submit.head()